### 1. Setup & Imports

In [1]:
# Install necessary packages 
import os
import pickle
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
import openai
from dotenv import load_dotenv

/Users/macbook/Documents/KULIAH/Project/AI_Shopping_Assistant/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/macbook/Documents/KULIAH/Project/AI_Shopping_Assistant/venv/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSF

### 2. Load Configuration

In [2]:
# Load environment variables from .env 
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "all-MiniLM-L6-v2"
METADATA_PATH = "../models/embeddings_faiss.pkl"
INDEX_PATH = "../models/faiss_index.bin"

# Setup OpenAI client
client = openai.OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL)


### 3. Chatbot Utility (ChatGPT Completion)

In [3]:
def chatbot_response(user_query):
    for attempt in range(3):  
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": user_query}],
                max_tokens=300,
                temperature=0.3
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error: {e}")
            break
    return "AI response unavailable."


### 4. Load Data, Model, and FAISS Index

In [4]:
# Load data & embeddings
df, embeddings = pickle.load(open(METADATA_PATH, "rb"))

# Normalize factors
max_price = df["actual_price"].max()
max_rating = df["ratings"].max()

# Load or rebuild FAISS index
if os.path.exists(INDEX_PATH):
    faiss_index = faiss.read_index(INDEX_PATH)
else:
    faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
    faiss_index.add(embeddings)
    faiss.write_index(faiss_index, INDEX_PATH)

# Load embedding model
embedding_model = SentenceTransformer(MODEL_NAME)


### 5. FAISS Search with Price & Rating Embedding

In [5]:
def search_faiss(query, price=0, rating=0, top_k=5):
    # Convert the search query into a numerical vector (embedding)
    query_embedding = embedding_model.encode([query], convert_to_numpy=True).astype(np.float32)
    
    # Normalize price & rating (avoid division by zero)
    normalized_price = np.array([[price / max_price]], dtype=np.float32) if max_price > 0 else np.array([[0]], dtype=np.float32)
    normalized_rating = np.array([[rating / max_rating]], dtype=np.float32) if max_rating > 0 else np.array([[0]], dtype=np.float32)

    # Make sure in the correct shape (1 row, 1 column)
    normalized_price = normalized_price.reshape(1, -1)
    normalized_rating = normalized_rating.reshape(1, -1)

    # Append price & rating to query embedding
    query_embedding = np.hstack((query_embedding, normalized_price, normalized_rating))
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Retrieve and format the top_k search results from the dataframe
    results = [
        {
            "name": df.iloc[idx]["name"],
            "actual_price": df.iloc[idx]["actual_price"],
            "ratings": df.iloc[idx]["ratings"],
            "score": float(distances[0][i])
        }
        for i, idx in enumerate(indices[0]) if idx < len(df)
    ]
    return results

### 6. Filter Utilities

In [6]:
def extract_price_range(query):
    query = query.lower()

    # Match patterns like "under ₹1000", "above ₹500", "below 200"
    match = re.search(r"(\bunder\b|\babove\b|\bbelow\b)?\s?[₹$]?\s?(\d+)", query, re.IGNORECASE)
    if match:
        condition = match.group(1) # 'under', 'above', 'below'
        price = int(match.group(2))

        if condition in ["under", "below"]:
            return ("max", price)
        elif condition == "above":
            return ("min", price)
        
    # Handle high-end products    
    if any(word in query for word in ["premium", "high-end", "luxury"]):
        return ("min", 60000)
    
    return None

### 7. Recommendation Flow

In [7]:
def recommend_products(user_query, top_k=5):
    """Recommend products using FAISS search with optional price filtering."""
    price_filter = extract_price_range(user_query)

    # Default price range
    min_price = 0
    max_price_filter = float("inf")
    
    # Apply extracted price filter
    if price_filter:
        filter_type, value = price_filter
        if filter_type == "max":
            max_price_filter = value
        elif filter_type == "min":
            min_price = value
    
    # Search FAISS
    results = search_faiss(user_query, price=min_price, top_k=top_k)

    # Filter strictly by price
    filtered = [r for r in results if min_price <= r["actual_price"] <= max_price_filter]
    
    return filtered


### 8. Summary Generator

In [12]:
def generate_summary(results):
    """Generate a structured summary ensuring name, price, and ratings are included."""

    # Check if the results list is empty
    if not results:
        return "No products to summarize."
    
    # Format each product's information
    product_info = "\n".join([
        f"- {item['name']} (Price: ₹{item['actual_price']}, Rating: {item['ratings']}⭐)"
        for item in results
    ])
    
    # Create a prompt for the chatbot
    prompt = (
        "Summarize these product recommendations in a friendly tone. "
        "Ensure each product includes its name, price (in ₹), and ratings. "
        "Show the product in a point list."
        "Do not omit any information:\n"
        f"{product_info}"
    )
    
    return chatbot_response(prompt)


### 9. Test Queries

In [18]:
# TEST: Try a Sample Query
test_queries = [
    "Air conditioner under ₹70000"
]

for q in test_queries:
    recommended = recommend_products(q)

    # # display retrieved data
    # print(f"🧾 Query: {q}")
    # for item in recommended:
    #     print(f"- {item['name']} | ₹{item['actual_price']} | ⭐ {item['ratings']}")
    
    print("📘 Summary:\n", generate_summary(recommended), "\n\n" + "="*100)



📘 Summary:
 - Stay cool this summer with these portable air conditioners!
1. Trends Alert Mini Air Conditioner Portable Air Mini Cooler Cooling Fan Mini Desktop Mobile Home Portable Air Conditioner (Price: ₹2999, Rating: 1.0⭐)
2. Go Arctic Air Portable 3 in 1 Conditioner Humidifier Purifier Mini Cooler Arctic Air Humidifier Purifier Mini Cooler Air (Price: ₹1899, Rating: 1.0⭐)
3. Haier 1.5 Ton 3 Star Inverter Split Air Conditioner Copper HSU19DW3DCINV White (Price: ₹69500, Rating: 1.0⭐)
4. Derike Portable Mini AC USB Battery Operated Air Conditioner Mini Water Air Cooler Cooling Fan Bladeless Dual Blower with (Price: ₹999, Rating: 1.0⭐)
5. Domact Mini AC USB Battery Operated Air Conditioner Mini Water Air Cooler Cooling Fan Bladeless Dual Blower with Ice Chamber (Price: ₹999, Rating: 1.0⭐) 

